<a href="https://colab.research.google.com/github/pinilDissanayaka/Skin-Diseases-Image-Classification/blob/main/Skin_Diseases_Using_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download haroonalam16/20-skin-diseases-dataset

 98% 313M/321M [00:02<00:00, 156MB/s]
100% 321M/321M [00:02<00:00, 127MB/s]


In [ ]:
! unzip /content/20-skin-diseases-dataset.zip

Archive:  /content/20-skin-diseases-dataset.zip
  inflating: Dataset/test/Acne and Rosacea Photos/07PerioralDermEye.jpg  
  inflating: Dataset/test/Acne and Rosacea Photos/07Rhinophyma1.jpg  
  inflating: Dataset/test/Acne and Rosacea Photos/07RosaceaFulFAce.jpg  
  inflating: Dataset/test/Acne and Rosacea Photos/07RosaceaMilia0120.jpg  
  inflating: Dataset/test/Acne and Rosacea Photos/07RosaceaOK0828063.jpg  
  inflating: Dataset/test/Acne and Rosacea Photos/07SteroidPerioral.jpg  
  inflating: Dataset/test/Acne and Rosacea Photos/07SteroidPerioral1.jpg  
  inflating: Dataset/test/Acne and Rosacea Photos/07VascularFace0120.jpg  
  inflating: Dataset/test/Acne and Rosacea Photos/07rhnophymas0321051.jpg  
  inflating: Dataset/test/Acne and Rosacea Photos/07sebDerem1101051.jpg  
  inflating: Dataset/test/Acne and Rosacea Photos/08PerioralDerm1.jpg  
  inflating: Dataset/test/Acne and Rosacea Photos/ComedonesEars-1.jpg  
  inflating: Dataset/test/Acne and Rosacea Photos/Sebaceous-glands-

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from glob import glob
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Rescaling, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

import warnings
warnings.filterwarnings(action = 'ignore')

In [ ]:
IMG_SIZE = [128, 128, 3]
basedir = '/content/Dataset'

In [ ]:
traindir = basedir + '/' + 'train'
testdir = basedir + '/' + 'test'

In [ ]:
print(f"Traing set has {len(os.listdir(traindir))} folders")
print(f"Testing set has {len(os.listdir(testdir))} folders")

Traing set has 19 folders
Testing set has 19 folders


In [ ]:
import shutil

shutil.rmtree('/content/Dataset/test/Bullous Disease Photos')

In [ ]:
traindatagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

testdatagen = ImageDataGenerator(
    rescale=1./255)

In [ ]:
traindf = traindatagen.flow_from_directory(
        traindir,
        target_size=(128, 128),
        batch_size=30,
        class_mode='categorical')

testdf = traindatagen.flow_from_directory(
        testdir,
        target_size=(128, 128),
        batch_size=30,
        class_mode='categorical')

Found 2609 images belonging to 19 classes.
Found 895 images belonging to 19 classes.


In [ ]:
labels = list(traindf.class_indices.keys())
outputsize = len(labels)

labels, outputsize

(['Acne and Rosacea Photos',
  'Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions',
  'Atopic Dermatitis Photos',
  'Cellulitis Impetigo and other Bacterial Infections',
  'Eczema Photos',
  'Exanthems and Drug Eruptions',
  'Herpes HPV and other STDs Photos',
  'Light Diseases and Disorders of Pigmentation',
  'Lupus and other Connective Tissue diseases',
  'Melanoma Skin Cancer Nevi and Moles',
  'Poison Ivy Photos and other Contact Dermatitis',
  'Psoriasis pictures Lichen Planus and related diseases',
  'Seborrheic Keratoses and other Benign Tumors',
  'Systemic Disease',
  'Tinea Ringworm Candidiasis and other Fungal Infections',
  'Urticaria Hives',
  'Vascular Tumors',
  'Vasculitis Photos',
  'Warts Molluscum and other Viral Infections'],
 19)

In [ ]:
model = Sequential()

# Inputs and rescaling
model.add(Rescaling(scale=1. / 255, input_shape=IMG_SIZE))

# Convolutional block 1
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=2))

# Convolutional block 2
model.add(Conv2D(128, (2, 2), activation='relu', padding='same'))
model.add(Conv2D(128, (2, 2), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=2))

# Convolutional block 3
model.add(Conv2D(256, (2, 2), activation='relu', padding='same'))
model.add(Conv2D(256, (2, 2), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=2))

# Convolutional block 4
model.add(Conv2D(512, (2, 2), activation='relu', padding='same'))
model.add(Conv2D(512, (2, 2), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=2))
model.add(BatchNormalization())
model.add(Flatten())

# Dense block
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=outputsize, activation='softmax'))


model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_2 (Rescaling)     (None, 128, 128, 3)       0         
                                                                 
 conv2d_120 (Conv2D)         (None, 128, 128, 64)      1792      
                                                                 
 conv2d_121 (Conv2D)         (None, 128, 128, 64)      36928     
                                                                 
 max_pooling2d_60 (MaxPooli  (None, 64, 64, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_122 (Conv2D)         (None, 64, 64, 128)       32896     
                                                                 
 conv2d_123 (Conv2D)         (None, 64, 64, 128)       65664     
                                                     

In [ ]:
earlystopping = EarlyStopping(monitor='val_loss', verbose=1, patience=1)

In [ ]:
model_history = model.fit_generator(
    traindf,
    epochs = 5,
    verbose = 1,
    validation_data = testdf,
    steps_per_epoch = len(traindf),
    validation_steps = len(testdf),
    callbacks = [earlystopping]
)

Epoch 1/5
87/87 [==============================] - 36s 366ms/step - loss: 2.4172 - accuracy: 0.3197 - val_loss: 2.5641 - val_accuracy: 0.2581
Epoch 2/5
87/87 [==============================] - 30s 340ms/step - loss: 2.3531 - accuracy: 0.3220 - val_loss: 2.5303 - val_accuracy: 0.2581
Epoch 3/5
87/87 [==============================] - 30s 349ms/step - loss: 2.3517 - accuracy: 0.3220 - val_loss: 2.5090 - val_accuracy: 0.2581
Epoch 4/5
87/87 [==============================] - 31s 354ms/step - loss: 2.3461 - accuracy: 0.3220 - val_loss: 2.5380 - val_accuracy: 0.2581
Epoch 4: early stopping
